In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
image_size = 224
batch_size = 48
num_classes = 2 
learning_rate = 0.001
num_epochs = 50

In [3]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
     transforms.Lambda(lambda x: x + 1.5 * torch.randn_like(x)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
output_folder = './split-dataset-2'
train_dataset = datasets.ImageFolder(os.path.join(output_folder, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(output_folder, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(output_folder, 'test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Modified ResNet50 model
class ModifiedResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet50, self).__init__()
        # Load pre-trained ResNet50 model
        self.resnet = models.resnet50(pretrained=True)
        # Replace the final layer with a modified fully connected layer
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_features, 512),  # Reduced dimensionality
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, num_classes)    # Output layer with num_classes
        )
    
    def forward(self, x):
        return self.resnet(x)

In [7]:
model = ModifiedResNet50(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

C:\Users\cse10\miniconda3\envs\data-science-env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\cse10\miniconda3\envs\data-science-env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
val_accuracies = []
train_losses = []
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch + 1}, Training Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")
    val_accuracies.append(accuracy)

print("Training complete.")

Epoch 1/50: 100%|████████████████████████████████████████████████████████████████████| 411/411 [02:48<00:00,  2.44it/s]


Epoch 1, Training Loss: 0.2470
Validation Accuracy: 92.13%


Epoch 2/50: 100%|████████████████████████████████████████████████████████████████████| 411/411 [02:41<00:00,  2.55it/s]


Epoch 2, Training Loss: 0.1747
Validation Accuracy: 93.96%


Epoch 3/50: 100%|████████████████████████████████████████████████████████████████████| 411/411 [02:41<00:00,  2.55it/s]


...

Epoch 48, Training Loss: 0.0604
Validation Accuracy: 97.06%


Epoch 49/50: 100%|███████████████████████████████████████████████████████████████████| 411/411 [02:39<00:00,  2.57it/s]


Epoch 49, Training Loss: 0.0575
Validation Accuracy: 97.20%


Epoch 50/50: 100%|███████████████████████████████████████████████████████████████████| 411/411 [02:36<00:00,  2.63it/s]


Epoch 50, Training Loss: 0.0599
Validation Accuracy: 95.95%
Training complete.


In [9]:
import pandas as pd
val_df = pd.DataFrame({
    "Epoch": list(range(1, num_epochs + 1)),
    "Validation Accuracy": val_accuracies,
    "Training Loss": train_losses
})

# Append to a CSV file (ensure this file name is unique for each model)
val_df.to_csv("model-4_val_accuracy.csv", mode='a', index=False, header=not pd.io.common.file_exists("model-4_val_accuracy.csv"))

In [10]:
# Evaluation on test set
model.eval()
true_labels = []
predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Test Accuracy: 0.9564
Precision: 0.9561
Recall: 0.9564
F1 Score: 0.9542


In [11]:
# In each model notebook, at the end of your metrics calculation, do this:
import pandas as pd

# Example metrics for Model 1
resnet_metrics = {
    "Model": "Modified ResNet-50",
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1,
}

metrics_df = pd.DataFrame([resnet_metrics])

# Append to CSV file
metrics_df.to_csv("model_metrics-2.csv", mode='a', index=False, header=not pd.io.common.file_exists("model_metrics-2.csv"))
